In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as data

In [6]:
import pandas_datareader.data as web
start = '2010-01-01'
end = '2019-12-31'

df = web.DataReader('AAPL', 'stooq', start, end)
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2019-12-31,71.0084,71.9246,70.9077,71.9176,1.030855e+08
2019-12-30,70.8938,71.6864,69.8549,71.3983,1.472306e+08
2019-12-27,71.3015,71.9974,70.5647,70.9755,1.494081e+08
2019-12-26,69.7592,71.0214,69.7303,71.0034,9.527219e+07
2019-12-24,69.7273,69.7762,69.2907,69.6226,4.948450e+07


In [7]:
df.tail()

,Open,High,Low,Close,Volume
Date,,,,,
2010-01-08,6.40586,6.45421,6.36458,6.45401,5.246856e+08
2010-01-07,6.44444,6.45421,6.36409,6.41085,5.588893e+08
2010-01-06,6.52639,6.55241,6.41553,6.42221,6.452991e+08
2010-01-05,6.53905,6.56358,6.49230,6.52639,7.055130e+08
2010-01-04,6.49958,6.53008,6.46597,6.51522,5.764890e+08


In [ ]:
df=df.reset_index()
df.head()

Dropping date and Adj Close column

In [ ]:
df = df.drop(['Date'], axis=1)
df.head()

In [ ]:
plt.plot(df.Close)

In [ ]:
df

In [ ]:
# ma100 is created for finding the mean of the 100 values and displaying them.
# for that rolling() function is used.

ma100 = df.Close.rolling(100).mean()
ma100

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))

plt.plot(df.Close,'black')
plt.plot(ma100,'g') # this is the mean of 100 values 

In [ ]:
ma200 = df.Close.rolling(200).mean()
ma200

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))

ma100 = df.Close.rolling(100).mean()
ma100

plt.plot(df.Close, 'b')
plt.plot(ma100, 'g')  # this is the mean of 100 values
plt.plot(ma200, 'r') # this is the mean of 200 values 

In [ ]:
df.shape

In [ ]:
# splitting the data into training and testing
import pandas as pd
data_training = pd.DataFrame(df['Close'][0:int(len(df)*0.70)])
data_testing = pd.DataFrame(df['Close'][int(len(df)*0.70):int(len(df))])

print("Used data for training: ", data_training.shape)
print("Used data for testing: ", data_testing.shape)

In [ ]:
data_training.head()

In [ ]:
data_testing.head()

For LSTM , we need to scale the data into the range of [-1,1] and **Scaling is required.**

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

data_training_array = scaler.fit_transform(data_training)
data_training_array

- - The above **data_training_array** needs to be divided into following **x_train** and **y_train** variable.

In [ ]:
import numpy as np

# The data is dependant on the previous result. (ma100) and (ma200)
# So, to calculate the next day value we train the following array.
# 100 days are x_train and 101 day is y_train and it goes on changing.

x_train = []
y_train = []


for i in range(100, data_training_array.shape[0]):
    x_train.append(data_training_array[i-100: i])
    y_train.append(data_training_array[i, 0])


x_train

# Now, converting x_train into numpy value, so that it can be provided to LSTM.
x_train, y_train = np.array(x_train), np.array(y_train)

# ML Model

In [ ]:
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential

In [ ]:

model = Sequential()
model.add(LSTM(units=50,
               activation='relu',
               return_sequences=True,
               input_shape=(x_train.shape[1], 1)
               )
          )
model.add(Dropout(0.2))


# Second Layer
model.add(LSTM(units=60,
               activation='relu',
               return_sequences=True,
               )
          )
model.add(Dropout(0.3))

# Third Layer
model.add(LSTM(units=80,
               activation='relu',
               return_sequences=True,
               )
          )
model.add(Dropout(0.4))

# Fourth Layer
model.add(LSTM(units=120,
               activation='relu',
               )
          )
model.add(Dropout(0.5))


# Dense Layer - to connect all the layers
model.add(Dense(units=1))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=50)

In [ ]:
model.save('keras_model.h5')

In [ ]:
data_testing.head()

In [ ]:
data_training.head()

In [ ]:
# We need previous 100 days data and we need to comapre and append the list.
past_100_days = data_training.tail(100)
final_df = past_100_days.append(data_testing, ignore_index=True)

In [ ]:
final_df.head()

In [ ]:
input_data = scaler.fit_transform(final_df)
input_data

In [ ]:
input_data.shape

In [ ]:
x_test = []
y_test = []

for i in range(100, input_data.shape[0]):
    x_test.append(input_data[i-100: i])
    y_test.append(input_data[i, 0])

In [ ]:
x_test, y_test = np.array(x_test), np.array(y_test)

print(x_test.shape)
print(y_test.shape)

In [ ]:
# Making Predictions

y_predicted = model.predict(x_test)

In [ ]:
y_predicted.shape

In [ ]:
y_test

In [ ]:
y_predicted

In [ ]:
scaler.scale_

In [ ]:
scale_factor = 1/0.06402036
y_predicted = y_predicted * scale_factor
y_test = y_test * scale_factor

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y_test, 'b', label="Original price")
plt.plot(y_predicted, 'r', label="Predicted price")
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()